# Insert Two Objects into a Video

## Dependencies
- Python ≥3.6 on Linux or Python ≥3.8 on Windows (which avoids WinError 6, see https://git.io/Jv8cN)
- moviepy
  - If you run into an error: `AttributeError: 'NoneType' object has no attribute 'stdout'` (see https://git.io/JvlyN) on moviepy version 1.0.1 try downgrading to version 1.0.0 of moviepy, explictly: `pip install moviepy==1.0.0`

## Usage
### Folder Structure

The script is expecting the following folder structure:

**🎥VideoFile.wmv**  
📂Object_FolderA_a  
├─ Object_File_1.png  
├─ Object_File_2.png  
📂Object_FolderA_b  
├─ Object_File_1.png  
├─ Object_File_2.png  
📂Object_FolderB_a  
├─ Object_File_1.png  
├─ Object_File_2.png  
📂Object_FolderB_b  
├─ Object_File_1.png  
├─ Object_File_2.png    
 
### Merging Strategy
In each iteration two objects will be placed in a fixed video file. One object will be placed on the left side and the other one one the right side. You can define the pixels later. Moreover, their positions will also be swapped.

For example, in the directory structure above, `Video-File.wmv` will be combined with `Object_File_1.png` and `Object_File_2.png` from folder `Object_FolderA_a` into a single video. Thereafter, `Video-File.wmv` gets merged with `Object_File_2.png` and `Object_File_2.png` from the second folder `Object_FolderA_b`, etc. The resulting video files will be stored in an `/out2` folder. When running the script using the above directory structure, the following output will be generated (including swapped positions):

📂\[other folders from above\]  
📂out2  
├─Object_FolderA_a-Object_File_1-left-Object_FolderA_b-Object_File_1-right.wmv  
├─Object_FolderA_a-Object_File_1-right-Object_FolderA_b-Object_File_1-left.wmv  
├─Object_FolderA_a-Object_File_2-left-Object_FolderA_b-Object_File_2-right.wmv  
├─Object_FolderA_a-Object_File_2-right-Object_FolderA_b-Object_File_2-left.wmv  
├─Object_FolderB_a-Object_File_1-left-Object_FolderB_b-Object_File_1-right.wmv  
├─Object_FolderB_a-Object_File_1-right-Object_FolderB_b-Object_File_1-left.wmv  
├─Object_FolderB_a-Object_File_2-left-Object_FolderB_b-Object_File_2-right.wmv  
├─Object_FolderB_a-Object_File_2-right-Object_FolderB_b-Object_File_2-left.wmv  

### ✏️ Edit and Adjust the Script
Sections containing a ✏️ icon can be adjusted to meet specific requirements.



## ▶ Generate Video Script
After installing `moviepy` (see above), start executing the script:

In [ ]:
import time
from pathlib import Path # pathlib > os.path.join (see http://tiny.cc/d1o1jz)
import moviepy.editor as mp

### ✏️ Folders
Define video and object folders.

In [ ]:
video_file = "PreferentialLookingSlide.wmv"
object_partner_1 = ["ObjectW_a", "ObjectX_a", "ObjectY_a", "ObjectZ_a"]
object_partner_2 = ["ObjectW_b", "ObjectX_b", "ObjectY_b", "ObjectZ_b"]

### ✏️ Parameters
*See `VideoObjectComposer.ipynb` for information about parameters*.

In [ ]:
object_left_x_pos = 400
object_left_y_pos = "center"
object_right_x_pos = 2048 - 260 - 400
object_right_y_pos = "center"
folder_file_delimter = "_"
object_extension = "png"
video_extension = "wmv" # wmv, avi
use_codec = "msmpeg4" # msmpeg4 (for wmv), libxvid (xvid for avi)
use_bitrate = None # CBR: "10M" # use this for constant
use_ffmpeg_params = ["-qscale", "2"] # VBR: ["-qscale", "2"]
use_preset = "ultrafast" # "ultrafast", default is medium
use_sound = False
output_folder = "out2"

### 📝 Generate File List & Queue



In [ ]:
# Create Partner 1 File List
object_partner_1_files = []
for op1_item in object_partner_1:
    couples1 = list(Path(op1_item).glob("**/*"))
    for single1 in couples1:
        object_partner_1_files.append(str(single1))

# Create Partner 2 File List
object_partner_2_files = []
for op2_item in object_partner_2:
    couples2 = list(Path(op2_item).glob("**/*"))
    for single2 in couples2:
        object_partner_2_files.append(str(single2))


# Check if video output folder is there, if not create it
Path("./" + output_folder).mkdir(parents=True, exist_ok=True)


### 🔁 Process Loop

In [ ]:
# start counting
start_time = time.time()

# Video Loop
video_counter = 0

partner_length = len(object_partner_1_files)
couples_length = partner_length * 2

# Video file is fixed
mp_obj1 = mp.VideoFileClip(video_file).set_duration(10)

print("Start Processing Videos")
for i in range(partner_length):

    video_counter += 1
    print(f"\nVideo {video_counter} of {couples_length}...")

    # Define Object Partner 1
    mp_obj2 = ( 
            mp.ImageClip(object_partner_1_files[i])
            .set_duration(mp_obj1.duration)
            .set_pos((object_left_x_pos, object_left_y_pos))
        )
    
    # Define Object Partner 2
    mp_obj3 = ( 
            mp.ImageClip(object_partner_2_files[i])
            .set_duration(mp_obj1.duration)
            .set_pos((object_right_x_pos, object_right_y_pos))
        )

    # Define Current Filename
    current_filename = object_partner_1_files[i].replace("/", folder_file_delimter).replace("\\", folder_file_delimter).replace("." + object_extension, folder_file_delimter) + "LEFT_" + object_partner_2_files[i].replace("/", folder_file_delimter).replace("\\", folder_file_delimter).replace("." + object_extension, "_RIGHT." + video_extension)
    
    # Create Composition
    mp_composition = mp.CompositeVideoClip([mp_obj1, mp_obj2, mp_obj3])
    mp_composition.write_videofile("./" + output_folder + "/" + current_filename, \
            codec=use_codec, bitrate=use_bitrate, ffmpeg_params=use_ffmpeg_params, audio=use_sound, preset=use_preset)
    # Closing the clip avoids Errno 12 (Cannot allocate memory)
    mp_composition.close()

    # --------------------------------------------------------
    # SWAP PROCESS Partner Positions
    # --------------------------------------------------------
    video_counter += 1
    print(f"\nVideo {video_counter} of {couples_length}...")
    
    print('SWAP')
    object_partner_1_files[i], object_partner_2_files[i] = object_partner_2_files[i], object_partner_1_files[i]
    print(object_partner_1_files[i],object_partner_2_files[i])

    # Define Object Partner 1
    mp_obj2 = ( 
            mp.ImageClip(object_partner_1_files[i])
            .set_duration(mp_obj1.duration)
            .set_pos((object_left_x_pos, object_left_y_pos))
        )
    
    # Define Object Partner 2
    mp_obj3 = ( 
            mp.ImageClip(object_partner_2_files[i])
            .set_duration(mp_obj1.duration)
            .set_pos((object_right_x_pos, object_right_y_pos))
        )

    # Define Current Filename
    current_filename = object_partner_1_files[i].replace("/", folder_file_delimter).replace("\\", folder_file_delimter).replace("." + object_extension, folder_file_delimter) + "LEFT_" + object_partner_2_files[i].replace("/", folder_file_delimter).replace("\\", folder_file_delimter).replace("." + object_extension, "_RIGHT." + video_extension)
    
    # Create Composition
    mp_composition = mp.CompositeVideoClip([mp_obj1, mp_obj2, mp_obj3])
    mp_composition.write_videofile("./" + output_folder + "/" + current_filename, \
            codec=use_codec, bitrate=use_bitrate, ffmpeg_params=use_ffmpeg_params, audio=use_sound, preset=use_preset)
    # Closing the clip avoids Errno 12 (Cannot allocate memory)
    mp_composition.close()



# End of Video Loop
print("💯 all done")
print(f"Execution time in seconds: {time.time() - start_time:.2f}")




